In [4]:
import json
from os import listdir
from os.path import isfile, join

def get_all_files_from_path(mypath):
    filenames = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
    return filenames
    
def readfile(filename):
    f = open(filename)
    data = json.load(f)
    return data

def get_avg_score(data):
    sum_score = 0
    for idx in data:
        sum_score += data[idx]
    return sum_score/len(data)
# get_avg_score(data)

def get_topk(list_prompt, data, avg_score):
    for idx in data:
        if data[idx] > avg_score:
            if idx not in list_prompt:
                list_prompt.update({idx: 1})
            else:
                list_prompt[idx] += 1
    return list_prompt

# list_prompt = {}
# avg = get_avg_score(data)
# get_topk(list_prompt, data, avg)

def get_best_prompt(list_prompt, files):
    for file in files:
        data = readfile(file)
        avg = get_avg_score(data)
        list_prompt = get_topk(list_prompt, data, avg)
    return list_prompt

from os import listdir
from os.path import isfile, join

def get_all_files_from_path(mypath):
    filenames = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
    return filenames

from bs4 import BeautifulSoup
import re
import json

def get_article(articles):
    result = {}
    current_statue = "(non-statute)"
    for i in re.split(r"(.*)", articles.strip()):
        if len(i) == 0 or i == "\n":
            continue
        if re.search(r"^\(.*\)$", i):
            current_statue = i.strip()
            if current_statue not in result:
                result.update({current_statue: []})
        else:
            if current_statue not in result:
                result.update({current_statue: []})
            result[current_statue].append(i)
    return result

def build_test(filename):
    result = {}
    with open(filename, 'r') as f:
        data = f.read()

    data = BeautifulSoup(data, "xml").find_all('pair')
    for i in data:
        id = i.get('id')
        result.update({id: {}})
        result[id].update({"label": i.get('label')})
        articles = i.find('t1').text.strip()
        # articles = get_article(articles)
        result[id].update({"result": articles})
        result[id].update({"content": i.find('t2').text.strip()})
    return result

def write_json(filename, data):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

import xml.etree.ElementTree as Et
import glob

def format_first_line(text):
    lines = text.split("\n")
    results = []
    for line in lines:
        if line[0] == "":
            continue
        if line[0] == "(" and line[-1] == ")":
            continue
        results.append(line)
    return "\n".join(results)

def load_samples(filexml):
    # try:
    tree = Et.parse(filexml)
    root = tree.getroot()
    samples = []
    for i in range(0, len(root)):
        sample = {'result': []}
        for j, e in enumerate(root[i]):
            if e.tag == "t1":
                sample['result'] = format_first_line(e.text.strip())
            elif e.tag == "t2":
                question = e.text.strip()
                sample['content'] = question if len(question) > 0 else None
        sample.update(
            {'index': root[i].attrib['id'], 'label': root[i].attrib.get('label', "N")})
        # filter the noise samples
        if sample['content'] is not None:
            samples.append(sample)
        else:
            print("[Important warning] samples {} is ignored".format(sample))
    return samples

def load_test_data_samples(path_folder_base, test_id):
    data = []
    test = load_samples(f"{path_folder_base}/riteval_{test_id}.xml")
    for file_path in glob.glob(f"{path_folder_base}/riteval_{test_id}.xml"):
        data = data + load_samples(file_path)
    return data


def load_all_data_samples(path_folder_base):
    data = []
    for file_path in glob.glob("{}/*.xml".format(path_folder_base)):
        data = data + load_samples(file_path)
    return data

def check_false_labels(pred, false_labels):
	for label in false_labels:
		if label in pred:
			return True
	return False

from tqdm import tqdm

def format_output(text):
	CLEANR = re.compile('<.*?>') 
	cleantext = re.sub(CLEANR, '', text)
	return cleantext.strip().lower()

def readfile(filename):
    f = open(filename)
    data = json.load(f)
    return data

In [2]:
path = "/home/congnguyen/drive/Coliee2024/output/accuracy/"

In [4]:
list_prompt = {}

for folder in ["alpaca", "t5-xxl"]:
    absolute_path = path + folder
    files = get_all_files_from_path(absolute_path)
    list_prompt = get_best_prompt(list_prompt, files)
# list_prompt

In [3]:
path_file = "../data/COLIEE2024statute_data-English/fewshot/"
get_all_files_from_path(path_file)

['../data/COLIEE2024statute_data-English/fewshot/riteval_H18_en.xml',
 '../data/COLIEE2024statute_data-English/fewshot/riteval_H30_en.xml',
 '../data/COLIEE2024statute_data-English/fewshot/riteval_H29_en.xml',
 '../data/COLIEE2024statute_data-English/fewshot/riteval_H25_en.xml',
 '../data/COLIEE2024statute_data-English/fewshot/riteval_H22_en.xml',
 '../data/COLIEE2024statute_data-English/fewshot/riteval_H21_en.xml',
 '../data/COLIEE2024statute_data-English/fewshot/riteval_R04_en.xml',
 '../data/COLIEE2024statute_data-English/fewshot/riteval_H24_en.xml',
 '../data/COLIEE2024statute_data-English/fewshot/riteval_H28_en.xml',
 '../data/COLIEE2024statute_data-English/fewshot/riteval_H20_en.xml',
 '../data/COLIEE2024statute_data-English/fewshot/riteval_R01_en.xml',
 '../data/COLIEE2024statute_data-English/fewshot/riteval_R02_en.xml',
 '../data/COLIEE2024statute_data-English/fewshot/riteval_H26_en.xml',
 '../data/COLIEE2024statute_data-English/fewshot/riteval_R03_en.xml',
 '../data/COLIEE2024

In [4]:
files = get_all_files_from_path(path_file)
train_articles = {}
for file in files:
    if "riteval_H" in file:
        data = load_samples(file)
        for item in data:
            primise = item["result"]
            articles = primise.split()[:2]
            print(articles)
            # for ar in articles:
            #     if ar not in train_articles:
            #         train_articles.update({ar: 0})
            #     else:
            #         train_articles[ar] += 1
# train_articles["Article 17"]

['Article', '572']
['Article', '565']
['Article', '568']
['Article', '697']
['Article', '698']
['Article', '698']
['Article', '702']
['Article', '697']
['Article', '715']
['Article', '715']
['Article', '175']
['Article', '295']
['Article', '329']
['Article', '398-20']
['Article', '398-3']
['Article', '398-2']
['Article', '398-7']
['Article', '398-12']
['Article', '9']
['Article', '9']
['Article', '7']
['Article', '152']
['Article', '169']
['Article', '537']
['Article', '537']
['Article', '107']
['Article', '255']
['Article', '254']
['Article', '485']
['Article', '446']
['Article', '465-2']
['Article', '465-4']
['Article', '465-3']
['Article', '419']
['Article', '404']
['Article', '192']
['Article', '3']
['Article', '34']
['Article', '34']
['Article', '101']
['Article', '103']
['Article', '94']
['Article', '111']
['Article', '102']
['Article', '113']
['Article', '115']
['Article', '116']
['Article', '169']
['Article', '176']
['Article', '177']
['Article', '201']
['Article', '180']
['Art

In [16]:
for file in files:
    if "riteval_R" in file:
        data = load_samples(file)
        print(file)
        count = 0
        list_articles = {}
        appeared_articles = {}
        for item in data:
            primise = item["result"]
            idx = item["index"]
            articles = re.findall(r'Article \d*', primise.replace("-", ""))
            for ar in articles:
                if ar in train_articles and ar != "Article ":
                    count += 1
                    break
        print(count)
        print(appeared_articles)
        print("==================================")
        

../data/COLIEE2024statute_data-English/fewshot/riteval_R04_en.xml
78
{}
../data/COLIEE2024statute_data-English/fewshot/riteval_R01_en.xml
100
{}
../data/COLIEE2024statute_data-English/fewshot/riteval_R02_en.xml
60
{}
../data/COLIEE2024statute_data-English/fewshot/riteval_R03_en.xml
69
{}


In [14]:
path = "../output/fewshot_detail/"
get_all_files_from_path("../output/fewshot_detail/")

[]

In [2]:
# import re

# print()


In [14]:

from os import listdir
from os.path import isfile, join

def get_all_files_from_path(mypath):
    filenames = [join(mypath, f) for f in listdir(mypath) if isfile(join(mypath, f))]
    return filenames

path = "../output/fewshot_detail/"


In [19]:
import xlsxwriter
 
# Workbook() takes one, non-optional, argument 
# which is the filename that we want to create.
workbook = xlsxwriter.Workbook('../output/fewshot_detail/result.xlsx')
worksheet = workbook.add_worksheet()
# worksheet.write('A1', NN)
worksheet.write('C1', "R04")
worksheet.write('G1', "R03")
worksheet.write('K1', "R02")
worksheet.write('O1', "R01")


0

In [20]:
def readtxt(file):
    f = open(file, "r", encoding="utf-8")
    data = f.readlines()
    return data

for i in range(1, 20):
    prompt = get_all_files_from_path("../output/fewshot_detail/prompt_"+str(i-1))
    for file in prompt:
        worksheet.write('B'+str(i+1), "prompt_"+str(i-1))
        NN = 0
        YY = 0
        YN = 0
        NY = 0
        if "acc" in file:
            data = readtxt(file)
            for line in data:
                if "\tN\tN" in line:
                    NN += 1
                elif "\tY\tY" in line:
                    YY += 1
                elif "\tY\tN" in line:
                    YN += 1
                elif "\tN\tY" in line:
                    NY += 1
            print(file)
            print(NN, " ", YY, " ", YN, " ",NY)
            if "04" in file:
                worksheet.write('C'+str(i+1), NN)
                worksheet.write('D'+str(i+1), YY)
                worksheet.write('E'+str(i+1), YN)
                worksheet.write('F'+str(i+1), NY)
            elif "03" in file:
                worksheet.write('G'+str(i+1), NN)
                worksheet.write('H'+str(i+1), YY)
                worksheet.write('I'+str(i+1), YN)
                worksheet.write('J'+str(i+1), NY)
            elif "02" in file:
                worksheet.write('K'+str(i+1), NN)
                worksheet.write('L'+str(i+1), YY)
                worksheet.write('M'+str(i+1), YN)
                worksheet.write('N'+str(i+1), NY)
            elif "01" in file:
                worksheet.write('O'+str(i+1), NN)
                worksheet.write('P'+str(i+1), YY)
                worksheet.write('Q'+str(i+1), YN)
                worksheet.write('R'+str(i+1), NY)

../output/fewshot_detail/prompt_0/riteval_R04_en_acc.txt
40   38   12   11
../output/fewshot_detail/prompt_0/riteval_R03_en_acc.txt
41   41   10   17
../output/fewshot_detail/prompt_0/riteval_R02_en_acc.txt
35   29   8   9
../output/fewshot_detail/prompt_0/riteval_R01_en_acc.txt
37   34   15   25
../output/fewshot_detail/prompt_1/riteval_R04_en_acc.txt
40   38   12   11
../output/fewshot_detail/prompt_1/riteval_R03_en_acc.txt
41   41   10   17
../output/fewshot_detail/prompt_1/riteval_R02_en_acc.txt
35   29   8   9
../output/fewshot_detail/prompt_1/riteval_R01_en_acc.txt
37   34   15   25
../output/fewshot_detail/prompt_2/riteval_R04_en_acc.txt
41   36   11   13
../output/fewshot_detail/prompt_2/riteval_R03_en_acc.txt
42   40   9   18
../output/fewshot_detail/prompt_2/riteval_R02_en_acc.txt
35   27   8   11
../output/fewshot_detail/prompt_2/riteval_R01_en_acc.txt
37   30   15   29
../output/fewshot_detail/prompt_3/riteval_R04_en_acc.txt
50   35   2   14
../output/fewshot_detail/prompt_

In [21]:
workbook.close()

# Result Check

In [23]:
def readtxt(file):
    f = open(file, "r", encoding="utf-8")
    data = f.readlines()
    return data

# r01, r02, r03, r04 = [], [], [], []
result = {}
labels = {}
for i in range(3, 12):
    # print(i)
    if i == 8:
        continue
    prompt = get_all_files_from_path("../output/fewshot_detail/prompt_"+str(i-1))
    for file in prompt:
        if "acc" in file:
            data = readtxt(file)
            for i in range(len(data)):
                id = data[i].strip().split("\t")[0]
                pred = data[i].strip().split("\t")[1]
                label = data[i].strip().split("\t")[2]
                if id not in result:
                    result.update({id: pred})
                else:
                    result[id] += " " + pred
                    
                if id not in labels:
                    labels.update({id: label})
# for k in result:
#     print(result[k], " - ", labels[k])

In [116]:
labels = {}
files = get_all_files_from_path("../data/COLIEE2024statute_data-English/test")

for file in files:
    data = load_samples(file)
    for item in data:
        labels.update({item["index"]: item["label"]})
labels     

{'H30-1-A': 'Y',
 'H30-2-I': 'N',
 'H30-2-U': 'N',
 'H30-4-A': 'Y',
 'H30-4-I': 'N',
 'H30-4-U': 'Y',
 'H30-4-E': 'N',
 'H30-4-O': 'Y',
 'H30-5-A': 'Y',
 'H30-5-U': 'N',
 'H30-5-E': 'Y',
 'H30-6-A': 'Y',
 'H30-7-A': 'Y',
 'H30-7-I': 'N',
 'H30-8-A': 'N',
 'H30-8-I': 'Y',
 'H30-9-A': 'N',
 'H30-9-I': 'Y',
 'H30-9-U': 'Y',
 'H30-11-U': 'Y',
 'H30-11-O': 'Y',
 'H30-12-E': 'Y',
 'H30-13-A': 'Y',
 'H30-13-I': 'Y',
 'H30-13-U': 'N',
 'H30-13-E': 'Y',
 'H30-15-A': 'N',
 'H30-15-I': 'Y',
 'H30-15-U': 'Y',
 'H30-15-E': 'N',
 'H30-15-O': 'Y',
 'H30-16-A': 'N',
 'H30-16-I': 'N',
 'H30-16-O': 'N',
 'H30-18-I': 'Y',
 'H30-18-U': 'N',
 'H30-18-E': 'N',
 'H30-19-A': 'N',
 'H30-19-I': 'N',
 'H30-19-U': 'Y',
 'H30-19-E': 'Y',
 'H30-20-A': 'N',
 'H30-20-E': 'N',
 'H30-20-O': 'N',
 'H30-22-A': 'N',
 'H30-22-I': 'N',
 'H30-22-U': 'Y',
 'H30-22-E': 'Y',
 'H30-22-O': 'Y',
 'H30-23-A': 'N',
 'H30-23-I': 'Y',
 'H30-23-O': 'Y',
 'H30-24-A': 'N',
 'H30-24-U': 'Y',
 'H30-24-E': 'Y',
 'H30-24-O': 'N',
 'H30-25-A'

In [175]:
def readtxt(file):
    f = open(file, "r", encoding="utf-8")
    data = f.readlines()
    return data

# r01, r02, r03, r04 = [], [], [], []
result = {}

prompt = get_all_files_from_path("../output/final/submit2")
for file in prompt:
    data = readtxt(file)
    print(file)
    for i in range(len(data)):
        id = data[i].strip().split(" ")[0]
        pred = data[i].strip().split(" ")[1]
        # label = data[i].strip().split("\t")[2]
        if id not in result:
            result.update({id: pred})
        else:
            result[id] += " " + pred
                
            # if id not in labels:
            #     labels.update({id: label})
check = {"H30": 0, "R01": 0, "R05": 0}
for k in result:
    print(k, ": ", result[k], " - ", labels[k])
check

../output/final/submit2/R03.task4.CAPTAIN3_riteval_R03_en.xml
../output/final/submit2/R03.task4.CAPTAIN1_riteval_R03_en.xml
../output/final/submit2/R03.task4.CAPTAIN2_riteval_R03_en.xml
R03-01-E :  N N N  -  N
R03-01-I :  Y Y Y  -  Y
R03-01-O :  Y N N  -  N
R03-02-A :  Y Y Y  -  Y
R03-02-E :  N Y N  -  Y
R03-02-I :  N N N  -  N
R03-02-O :  N Y N  -  N
R03-03-A :  Y Y Y  -  Y
R03-03-E :  Y Y Y  -  N
R03-03-I :  Y Y Y  -  Y
R03-03-O :  N N N  -  N
R03-03-U :  Y Y Y  -  Y
R03-04-A :  N Y Y  -  N
R03-04-E :  N Y N  -  N
R03-04-I :  N Y Y  -  Y
R03-04-O :  N Y Y  -  Y
R03-04-U :  N Y Y  -  Y
R03-05-A :  N N N  -  N
R03-05-O :  N N N  -  N
R03-05-U :  Y Y N  -  Y
R03-07-A :  Y Y Y  -  N
R03-07-E :  N N Y  -  N
R03-07-I :  Y Y Y  -  Y
R03-07-O :  N Y N  -  Y
R03-08-A :  Y N N  -  N
R03-08-E :  Y Y Y  -  Y
R03-08-O :  Y Y Y  -  Y
R03-08-U :  Y Y Y  -  Y
R03-09-A :  Y Y Y  -  Y
R03-09-E :  Y Y Y  -  Y
R03-09-U :  Y Y Y  -  N
R03-10-A :  N N N  -  N
R03-10-O :  N N N  -  Y
R03-10-U :  N Y Y  -  

{'H30': 0, 'R01': 0, 'R05': 0}

In [174]:
count_false = {"R01": 0, "R02": 0, "H30": 0, "R05": 0}
count_true = {"R01": 0, "R02": 0, "H30": 0, "R05": 0}
count = 0
for k in result:
    line = result[k].split(" ")
    # print(line)
    answer = line[0]
    # if line[0]!=line[1]:
    #     answer = line[0]
    #     # print(k, ": ", line[0], "-", line[1])
    #     if "R01" in k or "R1" in k: count_false["R01"] += 1
    #     elif "R02" in k or "R2" in k: count_false["R02"] += 1
    #     elif "H" in k or "R3" in k: count_false["H30"] += 1
    #     elif "R04" in k or "R4" in k: count_false["R04"] += 1
    if labels[k] == answer:
        if "R01" in k or "R1" in k: count_true["R01"] += 1
        elif "R02" in k or "R2" in k: 
            count_true["R02"] += 1
        elif "H" in k or "R3" in k: count_true["H30"] += 1
        elif "R05" in k or "R5" in k: count_true["R05"] += 1
count_true["R02"]/81

0.7901234567901234

In [169]:

count = 0
for k in result:
    N, Y = 0, 0
    text = result[k].split(" ")
    for i in text:
        if i == "N": N+=1
        else: Y+= 1
    answer = "N"
    if N < Y:
        answer = "Y"
    if labels[k] == answer:
        count += 1
        
count/81            

0.5802469135802469

In [145]:

count = 0
for k in result:
    text = result[k].split(" ")
    answer = text[0]
    if text[0] == "Y":
        answer = text[2]
    if labels[k] == answer:
        count += 1 
    # print(k, ": ", result[k], " - ", labels[k], " * ", answer)
count/81            

0.8271604938271605

In [90]:
count_false = {"R01": 0, "R02": 0, "H30": 0, "R04": 0}
count_true = {"R01": 0, "R02": 0, "H30": 0, "R04": 0}
for k in result:
    line = result[k].split(" ")
    # print(line)
    answer = line[1]
    if line[0]!=line[1]:
        answer = line[0]
        # print(k, ": ", line[0], "-", line[1])
        if "R01" in k or "R1" in k: count_false["R01"] += 1
        elif "R02" in k or "R2" in k: count_false["R02"] += 1
        elif "H" in k or "R3" in k: count_false["H30"] += 1
        elif "R04" in k or "R4" in k: count_false["R04"] += 1
    if labels[k] == answer:
        if "R01" in k or "R1" in k: count_true["R01"] += 1
        elif "R02" in k or "R2" in k: count_true["R02"] += 1
        elif "H" in k or "R3" in k: count_true["H30"] += 1
        elif "R04" in k or "R4" in k: count_true["R04"] += 1
count_true

{'R01': 0, 'R02': 0, 'H30': 45, 'R04': 0}

In [35]:
r01, r02, r03, r04 = 0, 0, 0 ,0
count = 0
for k in result:
    N, Y = 0, 0
    text = result[k].split()
    for i in text:
        if i == "N": N+=1
        else: Y+= 1
    answer = "N"
    if N <= Y:
        answer = "Y"
    if abs(N-Y) > 5 and labels[k] != answer:
        print(k)
    if ("R01" in k or "R1" in k) and labels[k] == answer: r01 += 1
    elif ("R02" in k or "R2" in k) and labels[k] == answer: r02 += 1
    elif ("R03" in k or "R3" in k) and labels[k] == answer: r03 += 1
    elif ("R04" in k or "R4" in k) and labels[k] == answer: r04 += 1
                

R04-03-U
R04-03-O
R04-04-A
R04-07-O
R04-08-O
R04-09-U
R04-16-U
R04-19-O
R04-20-A
R04-23-E
R04-24-U
R04-24-O
R03-02-E
R03-03-A
R03-03-E
R03-04-I
R03-04-U
R03-08-U
R03-09-U
R03-10-U
R03-13-A
R03-15-A
R03-16-O
R03-19-I
R03-20-E
R03-20-I
R03-21-I
R03-21-U
R03-23-O
R03-24-A
R03-27-I
R03-29-I
R03-36-U
R03-37-I
R02-1-A
R02-3-O
R02-4-U
R02-8-E
R02-9-I
R02-15-O
R02-16-I
R02-16-U
R02-16-E
R02-17-U
R02-17-O
R02-19-A
R02-24-A
R02-24-U
R02-29-A
R01-1-I
R01-1-U
R01-2-I
R01-2-U
R01-3-A
R01-3-E
R01-3-O
R01-4-O
R01-5-O
R01-6-A
R01-7-I
R01-7-E
R01-7-O
R01-8-A
R01-8-O
R1-13-O
R1-14-I
R1-15-I
R1-18-U
R1-19-E
R1-20-I
R1-21-I
R1-21-U
R1-22-I
R1-22-E
R1-23-5
R01-25-I
R01-26-A
R01-26-O
R01-28-I
R01-28-O
R01-36-U
R01-37-A
R01-37-I


In [58]:
count/111

0.0

In [36]:
print(r01)
print(r02)
print(r03)
print(r04)


69
64
77
84


In [5]:
f1 = open("../output/GPT_Llama2/prompt_4/gpt.txt", "r")
f2 = open("../data/COLIEE2024statute_data-English/test/answer-task4", "r")
count = 0
for l1, l2 in zip(f1, f2):
    l1=l1.strip()
    l2=l2.strip().split("\t")[1]
    if l1 == l2:
        count += 1
    
count 

85

In [6]:
85/109

0.7798165137614679